# Import Library

In [1]:
!pip install pymysql
!pip install xmltodict

In [2]:
import json
import urllib
import pandas as pd
from urllib.parse import urlencode, quote_plus, unquote, quote
from urllib.request import urlopen
from pandas.io.json import json_normalize
from sqlalchemy import create_engine
import pymysql
import datetime as dt
import sklearn
from sklearn.preprocessing import LabelEncoder
import json
import xmltodict

pymysql.install_as_MySQLdb()
import MySQLdb

pd.options.display.max_colwidth = 100

# Dataframe make function

In [3]:
service_key = 'SE0dHlfRrFs9F7Z8K6y6qmYRqrAFPtIrJnMdsrJLTMm1y5JvTVKl5Z8JpG4%2BBcvj%2Bv9kS1d1%2B8gQBmmxcMqMXg%3D%3D'

## Election Code dataframe make function

In [4]:
def get_election_df(pg_num = 1, num_row = 100):
  election_base_url = 'http://apis.data.go.kr/9760000/CommonCodeService/getCommonSgCodeList'

  page_no = str(pg_num)
  num_of_rows = str(num_row)                                  # maximum 34 datas

  queryParams = '?' + urlencode({
      quote_plus('pageNo') : page_no,
      quote_plus('numOfRows') : num_of_rows,
      quote_plus('resultType') : 'json',
      quote_plus('ServiceKey') : service_key 
  })

  API_election_code_url = election_base_url + unquote(queryParams)

  # Election code load

  '''
  < election code >
  (0)대표선거명               
  (1)대통령,(2)국회의원,(3)시도지사,(4)구시군장,(5)시도의원,(6)구시군의회의원
  (7)국회의원비례대표,(8)광역의원비례대표,(9)기초의원비례대표,(10)교육의원,(11)교육감
  '''

  response = urlopen(API_election_code_url)
  json_str = response.read()

  json_object = json.loads(json_str)

  body = [json_object['getCommonSgCodeList']['item']]

  election_code_data = pd.json_normalize(json_object['getCommonSgCodeList']['item'])
  
  return election_code_data

## Candidate Info dataframe make function

In [5]:
def get_candidate_df(sgId, sgTypecode, num_of_row=1000):
  candidate_base_url = 'http://apis.data.go.kr/9760000/PofelcddInfoInqireService/getPofelcddRegistSttusInfoInqire'

  queryParams = '?' + urlencode({
      quote_plus('numOfRows') : str(num_of_row),        # optional
      quote_plus('sgId') : str(sgId),                   # load from election code
      quote_plus('sgTypecode') : str(sgTypecode),       # load from election code
      quote_plus('sggName') : '',                       # optional
      quote_plus('sdName') : '',                        # optional
      quote_plus('resultType') : 'json',                # must set JSON
      quote_plus('ServiceKey') : service_key
  })

  API_candidate_url = candidate_base_url + unquote(queryParams)

  # candidate info load

  response = urlopen(API_candidate_url)
  json_str = response.read().decode('utf-8')

  json_object = json.loads(json_str)

  body = [json_object['getPofelcddRegistSttusInfoInqire']['item']]

  candidate_data = pd.json_normalize(json_object['getPofelcddRegistSttusInfoInqire']['item'])

  candidate_data.rename(columns={'HUBOID' : 'cnddtId', 'JD_NAME': 'partyName'}, inplace=True)

  return candidate_data

## Elction Promise dataframe make function

In [6]:
def get_promise_df(sgId, sgTypecode, hudoId):
  # 선거종료 후 데이터가 갱신되는 시점부터는 당선인의 공약 정보만 제공

  promise_base_url = 'http://apis.data.go.kr/9760000/ElecPrmsInfoInqireService/getCnddtElecPrmsInfoInqire'

  queryParams = '?' + urlencode({
      quote_plus('sgId') : str(sgId),                   # load from election code
      quote_plus('sgTypecode') : str(sgTypecode),       # load from election code
      quote_plus('cnddtId') : str(hudoId),              # parameter name is cnddtId but DB column name is HUBOID
      quote_plus('resultType') : 'json',                # must set JSON
      quote_plus('ServiceKey') : service_key
  })

  API_promise_url = promise_base_url + unquote(queryParams)

  # promise load

  response = urlopen(API_promise_url)
  json_str = response.read().decode('utf-8')

  json_object = json.loads(json_str)

  body = [json_object['getCnddtElecPrmsInfoInqire']['item']]

  promise_data = pd.json_normalize(json_object['getCnddtElecPrmsInfoInqire']['item'])

  return promise_data

## Polls Info dataframe make function


### Pre polls info dataframe make function

In [7]:
def get_prepoll_df(sgId, sdName):
  prepoll_base_url = 'http://apis.data.go.kr/9760000/PolplcInfoInqireService2/getPrePolplcOtlnmapTrnsportInfoInqire'

  queryParams = '?' + urlencode({
      quote_plus('sgId') : str(sgId),                   # load from election code
      quote_plus('sdName') : quote(sdName),             # must input parameter, query parma is korean, so wrap the quote
      quote_plus('resultType') : 'json',                # must set JSON
      quote_plus('ServiceKey') : service_key
  })

  API_prepoll_url = prepoll_base_url + unquote(queryParams)

  response = urlopen(API_prepoll_url)
  json_str = response.read().decode('utf-8')

  json_object = json.loads(json_str)

  body = [json_object['getPrePolplcOtlnmapTrnsportInfoInqire']['item']]

  prepoll_data = pd.json_normalize(json_object['getPrePolplcOtlnmapTrnsportInfoInqire']['item'])
  return prepoll_data

### Main polls info dataframe make function

In [8]:
def get_mainpoll_df(sgId, sdName):
  poll_base_url = 'http://apis.data.go.kr/9760000/PolplcInfoInqireService2/getPolplcOtlnmapTrnsportInfoInqire'

  queryParams = '?' + urlencode({
      quote_plus('sgId') : str(sgId),                   # load from election code
      quote_plus('sdName') : quote(sdName),             # must input parameter, query parma is korean, so wrap the quote
      quote_plus('resultType') : 'json',                # must set JSON
      quote_plus('ServiceKey') : service_key
  })

  API_poll_url = poll_base_url + unquote(queryParams)

  response = urlopen(API_poll_url)
  json_str = response.read().decode('utf-8')

  json_object = json.loads(json_str)

  body = [json_object['getPolplcOtlnmapTrnsportInfoInqire']['item']]

  poll_data = pd.json_normalize(json_object['getPolplcOtlnmapTrnsportInfoInqire']['item'])
  return poll_data

## Politician dataframe make function

### Current Politician

In [9]:
def get_curr_politician_mem(numOfRows=1000):
  curr_politician_base_url = 'http://apis.data.go.kr/9710000/NationalAssemblyInfoService/getMemberCurrStateList'

  queryParams = '?' + urlencode({
      quote_plus('numOfRows') : numOfRows,
      quote_plus('resultType') : 'json',   
      quote_plus('ServiceKey') : service_key
  })

  API_curr_politician_url = curr_politician_base_url + unquote(queryParams)

  response = urlopen(API_curr_politician_url)
  json_str = response.read().decode('utf-8')
  json_str = json.dumps(xmltodict.parse(json_str), indent=4)

  json_object = json.loads(json_str)

  curr_politician_data = pd.json_normalize(json_object['response']['body']['items']['item'])

  return curr_politician_data

In [10]:
get_curr_politician_mem().head()

,deptCd,empNm,engNm,hjNm,jpgLink,num,origNm,reeleGbnNm
0,9771230,강기윤,KANG GIYUN,姜起潤,http://www.assembly.go.kr/photo/9771230.jpg,2685,경남 창원시성산구,재선
1,9771142,강대식,KANG DAESIK,姜大植,http://www.assembly.go.kr/photo/9771142.jpg,3050,대구 동구을,초선
2,9771174,강득구,KANG DEUKGU,姜得求,http://www.assembly.go.kr/photo/9771174.jpg,3085,경기 안양시만안구,초선
3,9771233,강민국,KANG MINKUK,姜旻局,http://www.assembly.go.kr/photo/9771233.jpg,3091,경남 진주시을,초선
4,9771283,강민정,KANG MINJUNG,姜旼姃,http://www.assembly.go.kr/photo/9771283.jpg,3019,비례대표,초선


### Member detail

In [12]:
def get_politician_detail(dept_code, numOfRows=1000):
  politician_detail_base_url = 'http://apis.data.go.kr/9710000/NationalAssemblyInfoService/getMemberDetailInfoList'

  queryParams = '?' + urlencode({
      quote_plus('numOfRows') : numOfRows,
      quote_plus('dept_cd') : dept_code,
      quote_plus('resultType') : 'json',   
      quote_plus('ServiceKey') : service_key
  })

  API_politician_detail_url = politician_detail_base_url + unquote(queryParams)

  response = urlopen(API_politician_detail_url)
  json_str = response.read().decode('utf-8')
  json_str = json.dumps(xmltodict.parse(json_str), indent=4)

  json_object = json.loads(json_str)

  politician_detail_data = pd.json_normalize(json_object['response']['body']['item'])

  return politician_detail_data

In [13]:
get_politician_detail(9771106)

,assemEmail,assemTel,bthDate,electionNum,empNm,engNm,hbbyCd,hjNm,memTitle,origNm,polyNm,reeleGbnNm,secretary,secretary2,shrtNm,staff
0,nyleetogether@gmail.com,02-784-3866,19521220,대,이낙연,LEE NAKYON,독서,李洛淵,더불어민주당 당대표 (2020.08~현재)\r\n제21대 서울 종로구 국회의원 (2020.04~현재)\r\n더불어민주당 코로나19 국난극복위원장 (2020.03~2020.0...,서울 종로구,더불어민주당,5선제21대,"김대경, 하정철","김효섭, 염시진, 이상민, 장윤수",외교통일위원회,"노창훈, 이제이"


# DataFrame load test

## Election code load

In [174]:
get_election_df().head()

,NUM,SG_ID,SG_NAME,SG_TYPECODE,SG_VOTEDATE
0,1,20160413,제20대 국회의원선거,0,20160413
1,2,20160413,국회의원선거,2,20160413
2,3,20160413,구·시·군의장선거,4,20160413
3,4,20160413,시·도의회의원선거,5,20160413
4,5,20160413,구·시·군의회의원선거,6,20160413


## Cadidate info load

In [ ]:
get_candidate_df(20210407, 3).head()

,NUM,SG_ID,SG_TYPECODE,cnddtId,SGG_NAME,SD_NAME,WIW_NAME,GIHO,GIHO_SANGSE,partyName,NAME,HANJA_NAME,GENDER,BIRTHDAY,AGE,ADDR,JOB_ID,JOB,EDU_ID,EDU,CAREER1,CAREER2,STATUS
0,1,20210407,3,100138295,서울특별시,서울특별시,,1,,더불어민주당,박영선,朴映宣,여,19600122,61,서울특별시 서대문구 연희로27나길,75,정당인,221,서강대학교 언론대학원 졸업 (문학석사),(전) 중소벤처기업부 장관,"(전) 17대,18대,19대,20대 국회의원",등록
1,2,20210407,3,100138272,서울특별시,서울특별시,,2,,국민의힘,오세훈,吳世勳,남,19610104,60,서울특별시 광진구 능동로4길,85,변호사,221,고려대학교 법학대학원 졸업(법학박사),"(전) 제33, 34대 서울특별시장",(전) 16대 국회의원,등록
2,3,20210407,3,100138291,서울특별시,서울특별시,,4,,국민의당,안철수,安哲秀,남,19620122,59,서울특별시 노원구 동일로242나길,75,정치인,221,서울대학교 대학원 의학박사,"(전)제19대, 제20대 국회의원",(현)국민의당 대표,사퇴
3,4,20210407,3,100138218,서울특별시,서울특별시,,6,,기본소득당,신지혜,申智惠,여,19871022,33,서울특별시 은평구 통일로72길,75,정당인,68,이화여자대학교 사회학과 졸업(복수전공 : 여성학),(현)기본소득당 상임대표,(전)포이동 인연공부방 교사,등록
4,5,20210407,3,100138226,서울특별시,서울특별시,,7,,국가혁명당,허경영,許京寧,남,19470713,73,서울특별시 종로구 북촌로5나길,225,강연업,68,한국방송통신대학교 법학과 법학사,"(전)15, 17대 대통령 후보 출마",(현)국가혁명당 명예대표,등록


## Promise load

In [ ]:
# 당선인이 나온 후엔 당선인 공약만 확인 가능
data = get_promise_df(20210407, 3, 100138272)
data

,num,sgId,sgTypecode,cnddtId,sggName,sidoName,wiwName,partyName,krName,cnName,prmsCnt,prmsOrd1,prmsRealmName1,prmsTitle1,prmmCont1,prmsOrd2,prmsRealmName2,prmsTitle2,prmmCont2,prmsOrd3,prmsRealmName3,prmsTitle3,prmmCont3,prmsOrd4,prmsRealmName4,prmsTitle4,prmmCont4,prmsOrd5,prmsRealmName5,prmsTitle5,prmmCont5,prmsOrd6,prmsRealmName6,prmsTitle6,prmmCont6,prmsOrd7,prmsRealmName7,prmsTitle7,prmmCont7,prmsOrd8,prmsRealmName8,prmsTitle8,prmmCont8,prmsOrd9,prmsRealmName9,prmsTitle9,prmmCont9,prmsOrd10,prmsRealmName10,prmsTitle10,prmmCont10
0,1,20210407,3,100138272,서울특별시,서울특별시,,국민의힘,오세훈,吳世勳,5,1,,스피드 주택공급,□ 목 표\n ○ 주택공급 활성화로 부동산 시장 안정화\n \n□ 이행방법\n ○ 1년내 서울시 도시계획규제 혁파 \n - 서울시 용적률 규제완화(주거지역 용적률 ...,2,,스피드 교통,□ 목 표\n ○ 교통소외지역 연결 및 편의증대\n \n□ 이행방법\n ○ 1년내 시민불편공사 완료\n - 다리공사 국내 최장기록(11년4개월) 달성한 월드...,3,,균형발전 서울,"□ 목 표\n ○ 동북권, 서남권, 서북권 등 권역별 일자리, 상업, 교통, 교육 등 핵심시설을 유치하여 지역균형발전 달성 \n \n□ 이행방법\n ○ 서북권: 신...",4,,1인가구 안심특별대책본부 설치,"□ 목 표\n ○ 안전, 질병, 빈곤(저소득), 외로움(우울), 주거 등 1인가구 5대 불안해소\n \n□ 이행방법\n ○ 시장 직속 임시조직 : 조례 개정,...",5,,청춘이 밥 먹여준다!,"□ 목 표\n ○ 취업교육, 취업과 창업, 자산형성, 내집마련으로 이어지는 실질적이고 전방위적인 터널탈출 프로젝트\n \n□ 이행방법\n ○ 4차산업형 청년 취업사관...",,,,,,,,,,,,,,,,,,,,


## Poll info load

### Pre poll info load

In [ ]:
get_prepoll_df(20210407, '서울특별시').head()

,NUM,SG_ID,EV_PS_NAME,SD_NAME,WIW_NAME,EMD_NAME,EV_ORDER,PLACE_NAME,ADDR,FLOOR
0,1,20210407,가락1동사전투표소,서울특별시,송파구,가락1동,1,"가락1동주민센터(3층, 대강당)",서울특별시 송파구 양재대로 925 (가락동),3층
1,2,20210407,가락2동사전투표소,서울특별시,송파구,가락2동,1,"가락2동주민센터(2층, 다목적실)",서울특별시 송파구 중대로20길 6 (가락동),2층
2,3,20210407,가락본동사전투표소,서울특별시,송파구,가락본동,1,"가락본동주민센터(2층, 다목적방)",서울특별시 송파구 송파대로28길 39 (가락동),2층
3,4,20210407,가리봉동사전투표소,서울특별시,구로구,가리봉동,1,"가리봉동주민센터(1층, 회의실)",서울특별시 구로구 우마2길 35 (가리봉동),1층
4,5,20210407,가산동사전투표소,서울특별시,금천구,가산동,1,가산동주민센터 문화관람실(2층),서울특별시 금천구 가산로 129 (가산동),2층


### Main poll info load

In [ ]:
get_mainpoll_df(20210407, '서울특별시').head()

,NUM,SG_ID,PS_NAME,SD_NAME,WIW_NAME,EMD_NAME,PLACE_NAME,ADDR,FLOOR
0,1,20210407,청룡동제1투,서울특별시,관악구,청룡동,"봉림중학교(1층, 무용실)",서울특별시 관악구 장군봉길 72 (봉천동),1층
1,2,20210407,청림동제1투,서울특별시,관악구,청림동,"청림동주민센터(1층, 주차장)",서울특별시 관악구 관악로40길 39 (봉천동),1층
2,3,20210407,청운효자동제1투,서울특별시,종로구,청운효자동,"청운초등학교(2층, 체육관)",서울특별시 종로구 자하문로 105 (청운동),2층
3,4,20210407,청파동제1투,서울특별시,용산구,청파동,"청파도서관·청파동자치회관 별관(3층, 대강당)",서울특별시 용산구 청파로93길 27 (서계동),3층이상
4,5,20210407,충현동제1투,서울특별시,서대문구,충현동,"충현동주민센터(2층, 대회의실)",서울특별시 서대문구 북아현로22길 38 (북아현동),2층


# Data Preprocessing

## election_code data preprocessing

In [59]:
def election_code_preprocessing():
  data = get_election_df()
  data = data.drop('NUM', axis=1)
  data.columns = ['sgId', 'sgName', 'sgTypecode', 'sgVotedate']
  election_code = data
  election_code = election_code.astype({'sgId':int, 'sgTypecode':int, 'sgName':str})
  election_code.set_index('sgId', inplace=True)

  return election_code

In [60]:
election_code = election_code_preprocessing()

In [ ]:
data = election_code.reset_index()

## Politician data preprocessing

In [26]:
def politician_preprocessing():
  # current politician data preprocessing
  curr_data_df = get_curr_politician_mem()
  curr_data = curr_data_df.drop(['engNm', 'hjNm', 'origNm', 'reeleGbnNm'], axis=1)
  curr_data.columns = ['deptCode', 'name', 'imageLink', 'num']

  # export dept_code for find detail politician
  dept_code = curr_data['deptCode'].unique().astype(int)

  # detail politician data preprocessing
  detail_df = pd.DataFrame()

  for deptCd in dept_code:
    detail_data = get_politician_detail(deptCd)
    detail_data['deptCode'] = deptCd
    detail_df = pd.concat([detail_df, detail_data], ignore_index=True)

  detail_data = detail_df.drop(['assemEmail','assemHomep','assemTel', 'electionNum', 'engNm', 'hjNm', 'reeleGbnNm', 'secretary', 'secretary2', 'shrtNm', 'staff', 'hbbyCd', 'examCd'], axis=1)
  detail_data.columns = ['birthday', 'name', 'memTitle', 'sggName' , 'partyName', 'deptCode']

  # column type match to database column
  curr_data = curr_data.astype({'deptCode':int})
  detail_data = detail_data.astype({'deptCode':int})

  # join the current politician and detail poloitician
  politician = pd.merge(curr_data, detail_data, how='left', on=['name', 'deptCode'])
  politician['age'] = dt.datetime.now().year - pd.to_datetime(politician['birthday']).dt.year - 1
  politician['birthday'] = pd.to_datetime(politician['birthday'], format='%Y-%m-%d')
  politician.set_index('deptCode', inplace=True)

  return politician

In [27]:
politician = politician_preprocessing()

## Candidate data preprocessing

In [48]:
def candidate_code_preprocessing():
  election_code = election_code_preprocessing()
  data = election_code.reset_index()  
  election_data = data[['sgId', 'sgTypecode']]

  candidate_df = pd.DataFrame()

  # concat all candidate df
  for i in range(len(election_data)):
    curr_id = election_data['sgId'][i]
    curr_code = election_data['sgTypecode'][i]
    curr_df = pd.DataFrame()

    if curr_code != 0:
      curr_df = get_candidate_df(curr_id, curr_code)
    
    candidate_df = pd.concat([candidate_df, curr_df], ignore_index=True)
  
  # candidate column change
  candidate = candidate_df.drop(['NUM', 'GIHO_SANGSE', 'HANJA_NAME', 'JOB_ID', 'EDU_ID', 'EDU', 'CAREER1', 'CAREER2', 'partyName', 'AGE'], axis=1)
  candidate.columns = ['sgId', 'sgTypecode', 'cnddtId', 'sggName', 'sdName', 'wiwName', 'giho', 'name', 'gender', 'birthday','address', 'job', 'status']

  # encoding data
  le = LabelEncoder()

  for category in ['gender', 'status']:
    candidate[category] = le.fit_transform(candidate[category])

  # giho null value
  candidate[candidate['giho'] == ''] = 1

  # type matchin to sql column type
  candidate = candidate.astype({'sgId':int, 'sgTypecode':int, 'cnddtId':int, 'gender':int, 'status':int, 'giho':int})
  candidate['birthday'] = pd.to_datetime(candidate['birthday'], format='%Y-%m-%d')
  candidate.set_index('cnddtId', inplace=True)

  return candidate

In [49]:
candidate = candidate_code_preprocessing()

## Polls data preprocessing

### pre_polls data preprocessing

In [64]:
def pre_poll_preprocessing():
  sdarr = candidate['sdName'].unique()
  datearr = election_code['sgVotedate'].unique()
  
  total = pd.DataFrame()
  for dt in datearr:
    for sd in sdarr:
      try:
        cur = get_prepoll_df(dt, sd)
        total = pd.concat([total, cur], ignore_index=True)
      except:
        print(sd, 'no propoll at', dt)
  
  total.drop(['NUM'], axis=1, inplace=True)
  total.columns = ['sgId', 'evPsName', 'sdName', 'wiwName', 'emdName', 'evOrder', 'placeName','address', 'floor']  
  total = total.astype({'sgId':int, 'evOrder':int})
  total.set_index('sgId', inplace=True)
  
  return total

In [66]:
pre_poll = pre_poll_preprocessing()
pre_poll.head()

전국 no propoll at 20160413
전국 no propoll at 20170509
전국 no propoll at 20180613
서울특별시 no propoll at 20190403
부산광역시 no propoll at 20190403
대구광역시 no propoll at 20190403
인천광역시 no propoll at 20190403
광주광역시 no propoll at 20190403
대전광역시 no propoll at 20190403
울산광역시 no propoll at 20190403
세종특별자치시 no propoll at 20190403
경기도 no propoll at 20190403
강원도 no propoll at 20190403
충청북도 no propoll at 20190403
충청남도 no propoll at 20190403
전라남도 no propoll at 20190403
제주특별자치도 no propoll at 20190403
전국 no propoll at 20190403
전국 no propoll at 20200415
대구광역시 no propoll at 20210407
인천광역시 no propoll at 20210407
광주광역시 no propoll at 20210407
대전광역시 no propoll at 20210407
세종특별자치시 no propoll at 20210407
강원도 no propoll at 20210407
경상북도 no propoll at 20210407
제주특별자치도 no propoll at 20210407
전국 no propoll at 20210407


,evPsName,sdName,wiwName,emdName,evOrder,placeName,address,floor
sgId,,,,,,,,
20160413,용신동사전투표소,서울특별시,동대문구,용신동,1,"용신동주민센터(5층, 회의실)",서울특별시 동대문구 천호대로27길 35 (용두동),5층
20160413,제기동사전투표소,서울특별시,동대문구,제기동,1,"제기동주민센터(4층, 다목적강당)",서울특별시 동대문구 약령시로 71 (제기동),4층
20160413,중림동사전투표소,서울특별시,중구,중림동,1,"중림동주민센터(3층, 대강당)",서울특별시 중구 서소문로6길 16 (중림동),3층
20160413,금호1가동사전투표소,서울특별시,중구,금호1가동,1,"금호1가동주민센터(4층, 다목적실)",서울특별시 성동구 행당로 15 (금호동1가),4층
20160413,금호2·3가동사전투표소,서울특별시,중구,금호2·3가동,1,"금호2·3가동주민센터(3층, 다목적방)",서울특별시 성동구 무수막18길 1 (금호동2가),3층


### main_polls data preprocessing

In [67]:
def main_poll_preprocessing():
  sdarr = candidate['sdName'].unique()
  datearr = election_code['sgVotedate'].unique()

  total = pd.DataFrame()
  for dt in datearr:
    for sd in sdarr:
      try:
        cur = get_mainpoll_df(dt, sd)
        total = pd.concat([total, cur], ignore_index=True)
      except:
        print(sd, 'no mainpoll at', dt)
      
  total.drop(['NUM'], axis=1, inplace=True)
  total.columns = ['sgId', 'PsName', 'sdName', 'wiwName', 'emdName', 'placeName','address', 'floor']  
  total = total.astype({'sgId':int})
  total.set_index('sgId', inplace=True)
  
  return total

In [68]:
main_poll = main_poll_preprocessing()

전국 no mainpoll at 20160413
전국 no mainpoll at 20170509
전국 no mainpoll at 20180613
서울특별시 no mainpoll at 20190403
부산광역시 no mainpoll at 20190403
대구광역시 no mainpoll at 20190403
인천광역시 no mainpoll at 20190403
광주광역시 no mainpoll at 20190403
대전광역시 no mainpoll at 20190403
울산광역시 no mainpoll at 20190403
세종특별자치시 no mainpoll at 20190403
경기도 no mainpoll at 20190403
강원도 no mainpoll at 20190403
충청북도 no mainpoll at 20190403
충청남도 no mainpoll at 20190403
전라남도 no mainpoll at 20190403
제주특별자치도 no mainpoll at 20190403
전국 no mainpoll at 20190403
전국 no mainpoll at 20200415
대구광역시 no mainpoll at 20210407
인천광역시 no mainpoll at 20210407
광주광역시 no mainpoll at 20210407
대전광역시 no mainpoll at 20210407
세종특별자치시 no mainpoll at 20210407
강원도 no mainpoll at 20210407
경상북도 no mainpoll at 20210407
제주특별자치도 no mainpoll at 20210407
전국 no mainpoll at 20210407


In [70]:
main_poll.head()

,PsName,sdName,wiwName,emdName,placeName,address,floor
sgId,,,,,,,
20160413,가락2동제1투,서울특별시,송파구,가락2동,"가락2동주민센터(1층, 민원실)",서울특별시 송파구 중대로20길 6 (가락동),1층
20160413,일원본동제1투,서울특별시,강남구,일원본동,"대왕중학교(본관2층, 나래도서관)",서울특별시 강남구 광평로20길 7(수서동),1층
20160413,일원1동제1투,서울특별시,강남구,일원1동,일원1동 주민센터(1층),서울특별시 강남구 양재대로55길 14(일원동),1층
20160413,일원2동제1투,서울특별시,강남구,일원2동,"일원2동 주민센터(1층, 민원실)",서울특별시 강남구 개포로 607(개포동),1층
20160413,수서동제1투,서울특별시,강남구,수서동,"수서동주민센터(2층, 회의실)",서울특별시 강남구 광평로 301-4(수서동),2층


# MySQL link to GCP

## GCP MySQL connection

In [42]:
host = '34.64.72.201'
user = 'root'
pw = 'root'
db = 'election'
port = 3306

try:
    conn = pymysql.connect(host=host, port=port, user=user, password=pw, db=db)
    engine = create_engine('mysql+mysqldb://'+user+':'+pw+'@'+host+':3306/'+db, encoding='utf-8')

except Exception as err:
    print('Connection Failed (Error msg : ' + str(err) + ')')
else:
    print('Connection  Success!')

Connection  Success!


In [43]:
connection = engine.connect()
connection

# Queries & data insert

## CREATE TABLE

In [ ]:
create_election_code = """CREATE TABLE election_code (  
    sgId INT NOT NULL,
    sgTypecode INT NOT NULL,
    sgName INT NOT NULL,
    sgVotedate TIMESTAMP NOT NULL,
    PRIMARY KEY (sgId, sgTypecode)
) default charset utf8;"""

In [ ]:
create_pre_polls = """CREATE TABLE pre_polls (  
    sgId INT NOT NULL,
    sdName VARCHAR(30) NOT NULL ,
    wiwName VARCHAR(30) NOT NULL ,
    emdName VARCHAR(50) NOT NULL ,
    evOrder INT NOT NULL ,
    evPsName VARCHAR(80) NOT NULL ,
    placeName VARCHAR(100) NOT NULL ,
    `address` VARCHAR(150) NOT NULL ,
    `floor` INT NOT NULL ,
    PRIMARY KEY(evPsName) ,
    FOREIGN KEY(sgId) REFERENCES election_code(sgId)
) default charset utf8;"""

In [ ]:
create_main_polls = """CREATE TABLE main_polls (  
    sgId INT NOT NULL,
    sdName VARCHAR(30) NOT NULL ,
    wiwName VARCHAR(30) NOT NULL ,
    emdName VARCHAR(50) NOT NULL ,
    PsName VARCHAR(80) NOT NULL ,
    placeName VARCHAR(100) NOT NULL ,
    `address` VARCHAR(150) NOT NULL ,
    `floor` INT NOT NULL ,
    PRIMARY KEY(PsName) ,
    FOREIGN KEY(sgId) REFERENCES election_code(sgId)
) default charset utf8;"""

In [ ]:
create_politician = """CREATE TABLE politician (
    num INT NOT NULL ,
    `name` VARCHAR(20) NOT NULL ,
    imageLink TEXT ,
    sggName VARCHAR(35) NOT NULL ,
    deptCode INT NOT NULL ,
    memTitle LONGTEXT ,
    birthday TIMESTAMP NOT NULL ,
    age INT NOT NULL ,
    PRIMARY KEY (num)
) default charset utf8;"""

In [ ]:
create_candidate = """CREATE TABLE candidate (
    cnddtId INT NOT NULL ,
    num INT NOT NULL ,
    sgId INT NOT NULL ,
    sgTypecode INT NOT NULL ,
    sggName VARCHAR(35) NOT NULL ,
    sdName VARCHAR(30) NOT NULL ,
    wiwName VARCHAR(30) NOT NULL ,
    giho INT NOT NULL ,
    `name` VARCHAR(20) NOT NULL ,
    gender TINYINT NOT NULL ,
    `address` VARCHAR(100) ,
    job VARCHAR(100) ,
    `status` TINYINT NOT NULL ,
    birthday TIMESTAMP NOT NULL ,
    PRIMARY KEY (cnddtId),
    FOREIGN KEY (num) REFERENCES politician (num),
    FOREIGN KEY (sgId, sgTypecode) REFERENCES election_code (sgId, sgTypecode)
) default charset utf8;"""

In [ ]:
create_promise = """CREATE TABLE promise (
    prmIdx INT AUTO_INCREMENT NOT NULL ,
    cnddtId INT NOT NULL ,
    prmOrd INT ,
    prmTitle VARCHAR(100) ,
    prmCont LONGTEXT,
    PRIMARY KEY (prmIdx),
    FOREIGN KEY (cnddtId) REFERENCES candidate (cnddtId)
);"""

## Data insert

### election_code insert

In [61]:
election_code.to_sql(name='election_code', con=engine, if_exists='append')

### Politician insert

In [156]:
politician.to_sql(name='politician', con=engine, if_exists='append')

### candidate insert

In [62]:
candidate.to_sql(name='candidate', con=engine, if_exists='append')

### pre_poll insert

In [ ]:
pre_poll.to_sql(name='pre_poll', con=engine, if_exists='append')

### main_poll insert

In [ ]:
main_poll.to_sql(name='main_poll', con=engine, if_exists='append')